In [ ]:
# AsciiTextExtractor is a helper class to convert binary exe file to ascii text.
# It preserves only characters with ascii codes 10 and 32-127.
# It has iterator to look for content starting and ending with specific text.
# This is used to search for XML opening and closing tags.
    
class AsciiTextExtractor:
    def __init__(self, content, beginTxt, endTxt):
        self.__beginTxt = beginTxt
        self.__endTxt = endTxt

        filteredContent = bytearray(
            filter(lambda x: x == 10 or (x >= 32 and x < 127), content)
        )
        self.__contentTxt = filteredContent.decode("ascii")

    def __iter__(self):
        self.__idxEnd = 0
        self.__idxBegin = 0
        return self

    def __next__(self):
        if self.__idxBegin == -1:
            raise StopIteration

        self.__idxBegin = self.__contentTxt.find(self.__beginTxt, self.__idxEnd)
        if self.__idxBegin == -1:
            raise StopIteration

        self.__idxEnd = self.__contentTxt.find(
            self.__endTxt, self.__idxBegin + len(self.__beginTxt)
        )
        if self.__idxEnd == -1:
            raise StopIteration

        return self.__contentTxt[self.__idxBegin : self.__idxEnd + len(self.__endTxt)]

In [ ]:
# GetMsDownloadLinks downloads MS SQL exe installer file (around 6 Mbytes) and search for all little XMLs from the binary file.
# Then it parses XMLs to extract download URLs and SHA1 hashes.

import requests
from xml.dom.minidom import parseString

def GetMsDownloadLinks(msInstallerExeUrl):
    print(f"Requesting {msInstallerExeUrl}")
    request = requests.get(msInstallerExeUrl, allow_redirects=True)
    print(f"Downloaded {request.url}")
    beginTxt = '<Manifest xmlns="http://schemas.datacontract.org/2004/07/InstallerEngine" xmlns:i="http://www.w3.org/2001/XMLSchema-instance">'
    endTxt = "</Manifest>"
    for txt in AsciiTextExtractor(request.content, beginTxt, endTxt):
        xml = parseString(txt)
        reliableFiles = xml.getElementsByTagName("ReliableFile")
        for reliableFile in reliableFiles:
            downloadRoot = reliableFile.getElementsByTagName("DownloadRoot")
            fileName = reliableFile.getElementsByTagName("FileName")
            fileHash = reliableFile.getElementsByTagName("FileHash")
            if downloadRoot and fileName and fileHash:
                downloadUrl = f"{downloadRoot[0].firstChild.nodeValue}/{fileName[0].firstChild.nodeValue}"
                sha1Hash = f"{fileHash[0].firstChild.nodeValue}"
                yield (downloadUrl, sha1Hash)

In [ ]:
# GetMsSqlDownloadLinks is a wrapper around GetMsDownloadLinks
# It knows the SQL binary links, removes duplicates and sorts the results.
# It also allows filtering, e.g. for specific language or edition.

import os

sqlExelinks = {
    2017: "https://go.microsoft.com/fwlink/?linkid=853016",  # SQL Server 2017
    2019: "https://go.microsoft.com/fwlink/?linkid=866662",  # SQL Server 2019
    2022: "https://go.microsoft.com/fwlink/?linkid=2215158", # SQL Server 2022
}

# supported versions: 2022, 2019, 2017
def GetMsSqlDownloadLinks(version=2022, searchFilter=""):
    sqlExeUrl = sqlExelinks[version]

    # add links to the set to get rid of duplicates
    msDownloadLinks = set(GetMsDownloadLinks(sqlExeUrl))
    # add links to the to list, to be able to sort
    sortedMsDownloadList = list(msDownloadLinks)
    # sort by filename (x[0] is url)
    sortedMsDownloadList.sort(key=lambda x: os.path.basename(x[0]))
    
    return filter(lambda x: searchFilter in x[0], sortedMsDownloadList)

In [ ]:
# Print links to the english versions of MSSQL Server 2022 installer
for link, sha1 in GetMsSqlDownloadLinks(2022, "ENU"):
    print(f"{link} {sha1=}")

In [ ]:
# Let's save all the links to files
def saveSqlLinks(filename, sqlLinks):
    with open(filename, "w") as f:
        for link, sha1 in sqlLinks:
            f.write(f"{link} {sha1=}\n")

sql2017links = GetMsSqlDownloadLinks(version=2017)
sql2019links = GetMsSqlDownloadLinks(version=2019)
sql2022links = GetMsSqlDownloadLinks(version=2022)

saveSqlLinks("sql-2017-links.txt", sql2017links)
saveSqlLinks("sql-2019-links.txt", sql2019links)
saveSqlLinks("sql-2022-links.txt", sql2022links)